Hal class definition

In [2]:
# Highly Adaptive Lasso

from sklearn.linear_model import Lasso, LassoCV, RidgeCV
import numpy as np
from data_generator import DataGenerator
from highly_adaptive_lasso import HAL


In [4]:
X,Y = DataGenerator.generate_data(100)
X_test, Y_test = DataGenerator.generate_data(10000)

In [ ]:
model = HAL()
model.fit(X,Y)


# this displays the alpha values that were used in the LassoCV
# NOTE: alpha refers to the regularization parameter (?)
# TODO: Ask Alejandro about this
model.lasso.alphas_

In [7]:

model.lasso.alpha_

0.01176450890413735

In [ ]:
model = HAL()
model.fit(X,Y)

In [ ]:
Ŷ = model.predict(X_test)

In [ ]:
np.mean((Y_test - Ŷ)**2)

1.354553701520526